In [7]:
from skopt.sampler import Lhs
import pandas as pd
import numpy as np
from py21cmemu import Emulator

np.random.seed(42)

In [22]:
def lhs_sampler(n_samples, num_rounds, boundaries, column_names):
    
    sampler = Lhs(criterion="maximin", iterations=10000)

    dataframes = []

    for i in range(1, num_rounds + 1):
        dataframes.append('df' + f'{i}')
        sampled = sampler.generate(boundaries, n_samples)
        dataframes[i - 1] = pd.DataFrame(sampled, columns = column_names)
        print(i)
    
    return pd.concat(dataframes,  ignore_index = True)


In [23]:
test_param = [-0.98454527, 0.84028646, -1.01608287, 0.03414988, 9.02499104, 0.45168016, 40.0, 500.0, 1.0]
keys = ['F_STAR10', 'ALPHA_STAR', 'F_ESC10', 'ALPHA_ESC', 'M_TURN', 't_STAR', 'L_X','NU_X_THRESH', 'X_RAY_SPEC_INDEX']

input_dict = {k:v for k, v in zip(keys, test_param)}

df = pd.DataFrame([input_dict])

boundaries = []
for value in df.iloc[0]:
    boundaries.append((value - abs(value) * 0.1, value + abs(value) * 0.1))


In [24]:
#Traning 5000 samples for 2 rounds, i.e. 10000 samples with 9 parameters each took 21 minutes and 41.2 seconds without any speedup algorithm on my laptop

n_samples = 10
num_rounds = 2

training_data2 = lhs_sampler(n_samples = n_samples, num_rounds = num_rounds, boundaries = boundaries, column_names = keys)


1
2


In [60]:
emu = Emulator()

In [25]:
dict_data2 = training_data2.to_dict('records')

In [71]:

normed_input_params, output, output_errors = emu.predict(dict_data2, verbose = True)

1/1 [==============================] - 5s 5s/step


In [52]:
training_data1.to_pickle('training_data1_pickle2.pkl')

In [28]:
pickle_data = pd.read_pickle('training_data1_pickle.pkl')
pickle_emu = pickle_data.to_dict('records')

In [49]:
training_data1.to_csv('training_data1_csv.csv', index = False)

In [29]:
csv_data = pd.read_csv('training_data1_csv.csv')
csv_emu = csv_data.to_dict('records')

In [34]:
training_data1.to_hdf('training_data1_hdf5.h5', mode = 'w', key = 'Set1')

In [30]:
hdf5_data = pd.read_hdf('training_data1_hdf5.h5', key = 'Set1')
hdf5_emu = hdf5_data.to_dict('records')

In [ ]:
xarray = training_data2.to_xarray()
xarray.to_zarr('training_data2_zarr.zarr')

In [ ]:
import xarray
zarr_data = xarray.open_zarr('training_data2_zarr.zarr')
zarr_dataframe = zarr_data.to_dataframe()
emu_input = zarr_dataframe.to_dict('records')

ALPHA_ESC             0.034062
ALPHA_STAR            0.827166
F_ESC10              -1.084051
F_STAR10             -1.064604
L_X                  39.620770
M_TURN                8.304448
NU_X_THRESH         491.120770
X_RAY_SPEC_INDEX      0.901842
t_STAR                0.414408
Name: 0, dtype: float64
{'ALPHA_ESC': 0.03406233248278839, 'ALPHA_STAR': 0.8271656138779035, 'F_ESC10': -1.0840511933398838, 'F_STAR10': -1.0646043881146356, 'L_X': 39.620770099508285, 'M_TURN': 8.304447871901528, 'NU_X_THRESH': 491.1207701313355, 'X_RAY_SPEC_INDEX': 0.9018417280796861, 't_STAR': 0.41440750208198335}
